# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions;
                      functionals=[:lda_x, :lda_c_vwn],
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770886340313                   -0.52    9.0    181ms
  2   -2.772149139154       -2.90       -1.32    1.0    108ms
  3   -2.772170297222       -4.67       -2.45    1.0    111ms
  4   -2.772170656032       -6.45       -3.17    1.0    114ms
  5   -2.772170722878       -7.17       -4.39    2.0    132ms
  6   -2.772170722939      -10.22       -4.49    1.0    118ms
  7   -2.772170723013      -10.13       -5.25    1.0    119ms
  8   -2.772170723014      -11.89       -5.42    1.0    177ms
  9   -2.772170723015      -12.10       -6.40    1.0    474ms
 10   -2.772170723015      -13.31       -6.61    2.0    134ms
 11   -2.772170723015      -14.75       -7.27    1.0    143ms
 12   -2.772170723015   +  -14.45       -7.87    1.0    118ms
 13   -2.772170723015      -13.86       -8.69    2.0    148ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ----

1.7735578493219664

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770782012463                   -0.52    9.0    185ms
  2   -2.772052281052       -2.90       -1.32    1.0    121ms
  3   -2.772082967923       -4.51       -2.54    1.0    110ms
  4   -2.772083379192       -6.39       -3.50    1.0    115ms
  5   -2.772083416508       -7.43       -3.93    2.0    133ms
  6   -2.772083417721       -8.92       -4.77    1.0    120ms
  7   -2.772083417806      -10.07       -5.24    2.0    540ms
  8   -2.772083417811      -11.39       -6.39    1.0    116ms
  9   -2.772083417811      -13.09       -7.31    2.0    132ms
 10   -2.772083417811   +  -14.18       -7.34    1.0    124ms
 11   -2.772083417811      -14.51       -8.02    1.0    133ms

Polarizability via ForwardDiff:       1.7725349511844861
Polarizability via finite difference: 1.7735578493219664
